### **Problem Statement: Do states with higher chronic disease burden and higher hospital readmissions also have higher inpatient treatment costs?**

- Working on a Population Health Financial Impact Analysis, using three major U.S. healthcare datasets (CMS + HRRP + CDC) to explore how population health outcomes relate to financial and hospital performance metrics.
- https://data.cms.gov/ (CMS Datasets)
- https://data.cms.gov/ (HRRP Dataset)
- https://chronicdata.cdc.gov/ (Chronic Dataset)
- Studying the relationship between population health, readmissions, and costs 
- Goal is state-level comparison (e.g., “Which states are expensive and unhealthy?”),